In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

import pandas as pd
import numpy as np

In [2]:
data = pd.read_csv("sample_data/california_housing_train.csv")
data.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value
0,-114.31,34.19,15.0,5612.0,1283.0,1015.0,472.0,1.4936,66900.0
1,-114.47,34.40,19.0,7650.0,1901.0,1129.0,463.0,1.8200,80100.0
2,-114.56,33.69,17.0,720.0,174.0,333.0,117.0,1.6509,85700.0
3,-114.57,33.64,14.0,1501.0,337.0,515.0,226.0,3.1917,73400.0
4,-114.57,33.57,20.0,1454.0,326.0,624.0,262.0,1.9250,65500.0


In [10]:
data_test = pd.read_csv("sample_data/california_housing_test.csv")
data_test.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value
0,-122.05,37.37,27.0,3885.0,661.0,1537.0,606.0,6.6085,344700.0
1,-118.30,34.26,43.0,1510.0,310.0,809.0,277.0,3.5990,176500.0
2,-117.81,33.78,27.0,3589.0,507.0,1484.0,495.0,5.7934,270500.0
3,-118.36,33.82,28.0,67.0,15.0,49.0,11.0,6.1359,330000.0
4,-119.67,36.33,19.0,1241.0,244.0,850.0,237.0,2.9375,81700.0


In [3]:
X = data.drop('median_house_value', axis=1)
y = data['median_house_value']

In [4]:
class Net(nn.Module):

  def __init__(self):
    super(Net, self).__init__()

    self.fc1 = nn.Linear(8, 100)
    self.fc2 = nn.Linear(100, 100)
    self.out = nn.Linear(100, 1)

  def forward(self, x):
    x = F.normalize(x)
    x = F.relu(self.fc1(x))
    x = F.relu(self.fc2(x))
    
    y = self.out(x)

    return y

In [5]:
net = Net()

In [6]:
X_tensor = torch.from_numpy(data.values[:, :-1]).float()
y_tensor = torch.from_numpy(data.values[:, -1]).float()

y_tensor = torch.log(y_tensor)

In [11]:
X_test_tensor = torch.from_numpy(data_test.values[:, :-1]).float()
y_test = data_test.values[:, -1]


In [7]:
import torch.optim as optim
optimizer = optim.Adam(net.parameters(), lr=0.001)

In [8]:
criterion = nn.MSELoss()

In [16]:
for i in range(1001):
  optimizer.zero_grad()
  prediction = net(X_tensor)
  loss = criterion(prediction, y_tensor)
  loss.backward()
  optimizer.step()

  prediction2 = torch.exp(net(X_tensor))
  loss = criterion(prediction2, torch.exp(y_tensor))
  print(i," de 1000")
  print("Monitor de loss:", torch.sqrt(loss))

/usr/local/lib/python3.6/dist-packages/torch/nn/modules/loss.py:445: UserWarning: Using a target size (torch.Size([17000])) that is different to the input size (torch.Size([17000, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


0  de 1000
Monitor de loss: tensor(140297.3906, grad_fn=<SqrtBackward>)
1  de 1000
Monitor de loss: tensor(136732.7969, grad_fn=<SqrtBackward>)
2  de 1000
Monitor de loss: tensor(134945.1562, grad_fn=<SqrtBackward>)
3  de 1000
Monitor de loss: tensor(134385.0156, grad_fn=<SqrtBackward>)
4  de 1000
Monitor de loss: tensor(134606.5312, grad_fn=<SqrtBackward>)
5  de 1000
Monitor de loss: tensor(135297.8750, grad_fn=<SqrtBackward>)
6  de 1000
Monitor de loss: tensor(136207.3438, grad_fn=<SqrtBackward>)
7  de 1000
Monitor de loss: tensor(137160.9844, grad_fn=<SqrtBackward>)
8  de 1000
Monitor de loss: tensor(138047.5938, grad_fn=<SqrtBackward>)
9  de 1000
Monitor de loss: tensor(138798.1875, grad_fn=<SqrtBackward>)
10  de 1000
Monitor de loss: tensor(139370.6875, grad_fn=<SqrtBackward>)
11  de 1000
Monitor de loss: tensor(139750.4688, grad_fn=<SqrtBackward>)
12  de 1000
Monitor de loss: tensor(139926.4531, grad_fn=<SqrtBackward>)
13  de 1000
Monitor de loss: tensor(139903.6250, grad_fn=<Sqr

In [17]:
prediction_test = torch.exp(net(X_test_tensor))

In [18]:
prediction_test_np = prediction_test.detach().numpy()

In [19]:
from sklearn.metrics import mean_squared_error
y_test = data_test.values[:, -1]
np.sqrt(mean_squared_error(y_test, prediction_test_np))

117355.70388876511